In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import calc_f1
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [2]:
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
from transformers import AutoTokenizer
from src import config
from src.model import Model
from src.dataset import Dataset
from src.util import get_preds
import melt as mt
import husky

In [3]:
mn = 'base.20211227-025600'
mn = 'base.m2.20211227-113522'
v = '0'
bs = 64

In [4]:
model_dir = f'../working/offline/{v}/{mn}'
argv = open(f'{model_dir}/command.txt').readline().strip().split()
FLAGS(argv)
config.init_()
config.init()
model = Model()

ic| config.py:54 in init_()
    FLAGS.backbone: '../input/allenai/longformer-base-4096'
ic| config.py:54 in init_()
    FLAGS.backbone: '../input/allenai/longformer-base-4096'
ic| config.py:63 in init()
    FLAGS.valid_files[:2]: ['../input/feedback-prize-2021/tfrecords/train/70.164.tfrec',
                            '../input/feedback-prize-2021/tfrecords/train/90.164.tfrec']
All model checkpoint layers were used when initializing TFLongformerModel.

All the layers of TFLongformerModel were initialized from the model checkpoint at ../input/allenai/longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.
ic| model.py:30 in __init__()- num_classes: 8


In [5]:
dataset = Dataset(files=FLAGS.valid_files)
num_insts = dataset.num_instances
num_insts

get_num_records:   0%|          | 0/19 [00:00<?, ?it/s]

3119

In [6]:
d = dataset.make_batch(bs)

In [7]:
model(next(iter(d))[0])
model.load_weights(f'{model_dir}/model.h5')

In [8]:
  strategy = mt.distributed.get_strategy()
  with strategy.scope():   
    res = model.infer(d, steps=dataset.num_steps, callbacks=[husky.TQDMProgressBar('infer')])

infer:   0%|          | 0/49 [00:00<?, ?it/s]

In [9]:
gezi.save(res, f'{model_dir}/valid.pkl')